In [1]:
from budget import Budget
budget = Budget()

In [2]:
budget.generate_bill()

In [3]:
budget.bills

,category,Title,recurrence,recur_date,amt
0,Paycheck,Paycheck - BK,bi-weekly,2021-02-04,-2241.21
1,Mortgage & Rent,Rent,monthly,2021-01-28,2200.00
2,Electric,Ameren,monthly,2021-02-10,100.00
3,Natural Gas,Spire,monthly,2021-02-19,60.00
4,Mobile Phone,Sprint,monthly,2021-02-09,104.17
5,Internet,Spectrum,monthly,2021-01-31,80.00
6,Motorcycle Insurance,Dairyland,yearly,2021-07-16,181.65
7,Gym,FitnessYourWay,monthly,2021-02-16,25.00
8,Spotify,Spotify,monthly,2021-02-23,10.69


In [7]:
budget.last_7_df

,date,charges,amt
0,2021-01-24,|,0
1,2021-01-25,|,0
2,2021-01-26,|,0
3,2021-01-27,|,0
4,2021-01-28,| Rent |,2200.0
5,2021-01-29,|,0
6,2021-01-30,|,0


In [6]:
budget.next_21_df

,date,charges,amt
7,2021-01-31,| Rent | Spectrum | Dairyland | Spotify |,2472.34
8,2021-02-01,| Rent | Dairyland | Spotify |,2392.34
9,2021-02-02,| Rent | Dairyland | Spotify |,2392.34
10,2021-02-03,| Rent | Dairyland | Spotify |,2392.34
11,2021-02-04,| Paycheck - BK | Rent | Dairyland | Spotif...,151.13
12,2021-02-05,| Rent | Dairyland | Spotify |,2392.34
13,2021-02-06,| Rent | Dairyland | Spotify |,2392.34
14,2021-02-07,| Rent | Dairyland | Spotify |,2392.34
15,2021-02-08,| Rent | Dairyland | Spotify |,2392.34
16,2021-02-09,| Rent | Sprint | Dairyland | Spotify |,2496.51


# To-Do's
- Remove charges that already happened
- Store budget information
- Create calendar of how to predict upcoming paychecks
- Make list/calendar of upcoming expenses
- Calculate remaining budget based off upcoming expenses
- Create budget analysis (regression and past month)
- Create graphs and visuals
- Machine Learning budget